# Set up and global variables

In [ ]:
import os

from pathlib import Path
from itertools import combinations

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from tqdm import tqdm
from scipy.stats import kendalltau

from src.prioritization import *

# input data
VERSION = '0.0.0'
DATASET_PATH = Path('data') / 'datasets' / f'ipython_{VERSION}'
MODEL_PATH = DATASET_PATH / 'trained_heuristics'

DATA_PARTITION = 'hold_out'

# output data
SURVEY_DATA_PATH = Path('labelling_app') / 'survey_data' / f'ipython_{VERSION}'
MODELLING_DATA_PATH = DATASET_PATH / 'hold_out_prioritization_scores'

# config
RANDOM_SEED = 42
SAMPLE_SIZE = 70
RESOLUTION = 300
DEBUG = True

# images
IMAGE_DIR = Path('images') / "survey_preparation"

os.makedirs(SURVEY_DATA_PATH, exist_ok=True)
os.makedirs(MODELLING_DATA_PATH, exist_ok=True)
os.makedirs(IMAGE_DIR, exist_ok=True)

***

# Loading data

In [ ]:
items = pd.read_csv(DATASET_PATH / 'items.csv', index_col=0)
defects = pd.read_csv(DATASET_PATH / f'defects.csv', index_col=0)

log = pd.read_csv(DATASET_PATH / DATA_PARTITION/ 'log.csv', index_col=0, parse_dates=['time'])
defect_log = pd.read_csv(DATASET_PATH / DATA_PARTITION / 'defect_log.csv', index_col=0)
defect_log.columns = defect_log.columns.astype(int)

In [ ]:
models = [
    TaskCommonModel,
    TaskCharacteristicModel,
    StudentFrequencyModel,
    StudentCharacteristicModel,
    StudentEncounteredBeforeModel,
    DefectMultiplicityModel,
    SeverityModel,
]

models = [model.load(MODEL_PATH / f'{model.get_model_name()}.pkl') for model in models]

models = {model.get_model_name(): model for model in models}

***

# Quality filtering

In [ ]:
def has_close_pair(row):
    """Check if there is a pair of values with difference two or less."""
    row_values = row.values
    return np.any(np.abs(row_values[:, None] - row_values) <= 2)

In [ ]:
# at least two defects
filtered = defect_log[(defect_log > 0).sum(axis=1) > 1]

In [ ]:
# at most difference of two in severity
filtered *= defects.loc[filtered.columns]['severity']
filtered = filtered[filtered.apply(has_close_pair, axis=1)]

In [ ]:
# apply the filter
defect_log = defect_log.loc[filtered.index]
log = log.loc[filtered.index]

***

# Sampling criteria

In [ ]:
# These defects will not be used to calculate the sampling statistics
IGNORE_FOR_CALCULATION = defects[defects['defect name'] == 'inappropriate whitespace: visible'].index

# Calculate sampling statistics and remember heuristic scores
stats = []
discrete_scores = []
continuous_scores = []

# Ensure log is sorted by time
log = log.sort_values(by='time')

for idx, submission in tqdm(log.iloc[:200].iterrows() if DEBUG else log.iterrows(), total=log.shape[0], desc="Calculating statistics for sampling"):
    defect_counts = defect_log.loc[idx]

    # Heuristic scores
    discrete_model_scores = {name: model.discretize(submission, defect_counts).dropna() for name, model in models.items()}
    continuous_model_scores = {name: model._calculate_scores(submission, defect_counts) for name, model in models.items()}

    # Save model scores
    for defect in defect_counts[defect_counts > 0].index:
        discrete_row = {"submission id": idx, "defect id": defect}
        continuous_row = discrete_row.copy()
        for name, scores in discrete_model_scores.items():
            discrete_row[name] = scores[defect]
            continuous_row[name] = continuous_model_scores[name][defect]
        discrete_scores.append(discrete_row)
        continuous_scores.append(continuous_row)

    # Ignore some defects
    discrete_model_scores = {
        name: scores.loc[~scores.index.isin(IGNORE_FOR_CALCULATION)]
        for name, scores in discrete_model_scores.items()
    }

    # Disagreement
    kendall_distances = []
    for left, right in combinations(discrete_model_scores.values(), 2):
        tau, _ = kendalltau(left, right, nan_policy='raise')
        kendall_distances.append(1 - tau)

    disagreement = np.nanmean(kendall_distances) if any(~np.isnan(kendall_distances)) else 0.0

    # Signal strength
    signal_strength = np.mean([np.ptp(values) for values in discrete_model_scores.values()])

    for model in models.values():
        model.update(submission, defect_counts)

    stats.append({
        "id": idx,
        "signal strength": signal_strength,
        "model disagreement": disagreement,
    })

stats = pd.DataFrame(stats).set_index("id")
discrete_scores = pd.DataFrame(discrete_scores)
continuous_scores = pd.DataFrame(continuous_scores)

***

# Survey sample

In [ ]:
min_score = 0.1
rng = np.random.default_rng(RANDOM_SEED)

# Normalize
scores = (stats - stats.min()) / (stats.max() - stats.min())

# Combine scores
scores['total'] = scores.sum(axis=1)
scores['total'] = np.where(scores['total'] < min_score, min_score, scores['total'])

# Stratify by topic
strata_labels = items['topic'].unique()
strata_counts = np.zeros(strata_labels.shape[0], dtype=int)
scores['strata'] = log.loc[scores.index, 'item'].map(items['topic'])

# Greedy sampling
sampled_ids = []
while len(sampled_ids) < SAMPLE_SIZE:
    # Least represented strata
    topic_idx = rng.choice(np.where(strata_counts == strata_counts.min())[0], size=1)
    # Collect eligible submissions
    eligible_ids = scores[scores['strata'] == strata_labels[topic_idx][0]].index.difference(sampled_ids)
    # Normalize scores into probabilities
    probabilities = scores.loc[eligible_ids, 'total'] / scores.loc[eligible_ids, 'total'].sum()
    # Sample, skipping if there are no eligible submissions
    try:
        sampled_ids.append(rng.choice(eligible_ids, replace=False, p=probabilities.values))
    except ValueError:
        pass
    strata_counts[topic_idx] += 1

***

# Analysis

In [ ]:
sample_stats = stats.loc[sampled_ids]

## Validation plots

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

axes[0].hist(sample_stats["signal strength"], bins=30, edgecolor="black")
axes[0].set_title("Distribution of Signal Strength")
axes[0].set_xlabel("Signal Strength")
axes[0].set_ylabel("Count")

axes[1].hist(sample_stats["model disagreement"], bins=30, edgecolor="black")
axes[1].set_title("Distribution of Model Disagreement")
axes[1].set_xlabel("Model Disagreement")
axes[1].set_ylabel("Count")

plt.tight_layout()
plt.savefig(IMAGE_DIR / "criterion_distribution.png", dpi=RESOLUTION)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

ax.scatter(
    sample_stats["signal strength"],
    sample_stats["model disagreement"],
    alpha=0.6,
    s=30,
    c=log.loc[sample_stats.index, "item"].astype("category").cat.codes,  # color by task
)

ax.set_xlabel("Signal Strength")
ax.set_ylabel("Model Disagreement")
ax.set_title("Signal Strength vs Model Disagreement")

plt.tight_layout()
plt.savefig(IMAGE_DIR / "signal_vs_disagreement.png", dpi=RESOLUTION)
plt.show()


## Coverage

In [ ]:
defect_counts = defect_log.loc[sample_stats.index].sum().sort_values(ascending=False)

In [ ]:
defect_counts = defect_log.loc[sample_stats.index].sum().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(defect_counts.index.astype(str), defect_counts.values)
ax.set_xlabel("Defect ID")
ax.set_ylabel("Number of Submissions")
ax.set_title("Coverage of Defects in Hold-Out Partition")
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig(IMAGE_DIR / "defect_coverage.png", dpi=RESOLUTION)
plt.show()

In [ ]:
task_counts = log.loc[sample_stats.index, "item"].value_counts().reindex(items.index).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(task_counts.index.astype(str), task_counts.values)
ax.set_xlabel("Task ID")
ax.set_ylabel("Number of Submissions")
ax.set_title("Coverage of Tasks in Hold-Out Partition")
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig(IMAGE_DIR / "task_coverage.png", dpi=RESOLUTION)
plt.show()


In [ ]:
topics = items['topic'].unique()
topic_counts = items.loc[log.loc[sample_stats.index, "item"], "topic"].value_counts().reindex(topics).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(topic_counts.index.astype(str), topic_counts.values)
ax.set_xlabel("Topic")
ax.set_ylabel("Number of Submissions")
ax.set_title("Coverage of Topics in Hold-Out Partition")
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig(IMAGE_DIR / "topic_coverage.png", dpi=RESOLUTION)
plt.show()


In [ ]:
fig, axes = plt.subplots(
    nrows=int(np.ceil(len(models.keys()) / 3)),
    ncols=3,
    figsize=(12, 3 * int(np.ceil(len(models.keys()) / 3))),
    constrained_layout=True
)

axes = axes.flatten()

for ax, col in zip(axes, models.keys()):
    has_1_5_scale = models[col].get_discretization_scale() == '1-5'
    ax.hist(discrete_scores[col].dropna(), bins=np.arange(0.5, 6.5, 1) if has_1_5_scale else np.arange(-2.5, 3.5, 1), edgecolor="black", rwidth=0.8)
    ax.set_title(col)
    ax.set_xticks([1, 2, 3, 4, 5] if has_1_5_scale else [-2, -1, 0, 1, 2])
    ax.set_xlabel("Score")
    ax.set_ylabel("Count")

# Remove empty subplots
for ax in axes[len(models.keys()):]:
    ax.axis("off")

plt.suptitle("Distribution of Heuristic Scores (All Defects)", fontsize=14, y=1.02)
plt.savefig(IMAGE_DIR / "heuristic_value_distributions.png", dpi=RESOLUTION, bbox_inches="tight")
plt.show()


## Storytelling

In [ ]:
# High and low agreement / signal strength examples

***

# Combine for the survey

In [ ]:
# --- Submissions ---
submission_df = (
    log.loc[sampled_ids]
    .merge(items[['name', 'instructions']], left_on='item', right_index=True)
    .rename(columns={
        'answer': 'submission',
        'name': 'task name',
    })
    [['submission', 'task name', 'instructions']]
)

# --- Defects ---
# Long format
melted_defects = (
    defect_log.loc[sampled_ids]
    .stack()
    .reset_index()
    .rename(columns={'index': 'submission id', 'level_1': 'defect id', 0: 'count'})
)
melted_defects = melted_defects[melted_defects['count'] > 0]

# Add metadata
defect_df = (
    melted_defects
    .merge(defects, left_on='defect id', right_index=True)
    [['submission id', 'defect id', 'defect name', 'description', 'code example', 'code fix example']]
    .rename(columns={'defect name': 'name'})
)

# Add heuristic scores
defect_df = defect_df.merge(
    discrete_scores,
    on=['submission id', 'defect id'],
    how='left'
)

# --- Heuristics ---
heuristics_df = []
for name, model in models.items():
    heuristics_df.append({
        'name': name,
        'description': model.get_model_description(),
        'scale': model.get_discretization_scale(),
        'interpretation': model.get_model_interpretation()
    })

heuristics_df = pd.DataFrame(heuristics_df).set_index('name')

In [ ]:
submission_df.to_csv(SURVEY_DATA_PATH / 'submissions.csv', sep=';', index_label='index')
defect_df.to_csv(SURVEY_DATA_PATH / 'defects.csv', sep=';', index_label='index')
heuristics_df.to_csv(SURVEY_DATA_PATH / 'heuristics.csv', sep=';', index_label='name')

***

# Combine for modelling

In [ ]:
discrete_scores.to_csv(MODELLING_DATA_PATH / 'discrete_scores.csv', sep=';', index_label='index')
continuous_scores.to_csv(MODELLING_DATA_PATH / 'continuous_scores.csv', sep=';', index_label='index')